In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('nlp_learn.db')

# data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
query = "SELECT id, title FROM books"
result_set = conn.execute(query).fetchall()
data = pd.DataFrame(result_set, columns = ['id', 'title'])

data_text = data[['title']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
len(documents)

671255

In [3]:
documents[:10]

,title,index
0,Things Fall Apart,0
1,Fairy tales,1
2,The Divine Comedy,2
3,The Epic Of Gilgamesh,3
4,The Book Of Job,4
5,One Thousand and One Nights,5
6,Njál's Saga,6
7,Pride and Prejudice,7
8,Le Père Goriot,8
9,"Molloy, Malone Dies, The Unnamable, the trilogy",9


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\salman.akhatar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [7]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [8]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 1:
            result.append(lemmatize_stemming(token))
    return result

In [9]:
doc_sample = documents[documents['index'] == 4130].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Jump', 'Start', 'Your', 'Gluten-Free', 'Diet!', 'Living', 'with', 'Celiac', '/', 'Coeliac', 'Disease', '&', 'Gluten', 'Intolerance']


 tokenized and lemmatized document: 
['jump', 'start', 'gluten', 'free', 'diet', 'live', 'celiac', 'coeliac', 'diseas', 'gluten', 'intoler']


In [10]:
processed_docs = documents['title'].map(preprocess)

In [11]:
processed_docs[:10]

0                   [thing, fall, apart]
1                          [fairi, tale]
2                        [divin, comedi]
3                      [epic, gilgamesh]
4                            [book, job]
5                      [thousand, night]
6                           [njál, saga]
7                      [pride, prejudic]
8                     [le, père, goriot]
9    [molloy, malon, die, unnam, trilog]
Name: title, dtype: object

In [12]:
dictionary = gensim.corpora.Dictionary(processed_docs)
len(dictionary)

203957

In [13]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 apart
1 fall
2 thing
3 fairi
4 tale
5 comedi
6 divin
7 epic
8 gilgamesh
9 book
10 job


In [14]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4130]

[(694, 1),
 (779, 1),
 (1336, 1),
 (1443, 1),
 (1844, 1),
 (1888, 1),
 (3103, 2),
 (4419, 1)]

In [16]:
bow_doc_4310 = bow_corpus[4130]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 694 ("jump") appears 1 time.
Word 779 ("live") appears 1 time.
Word 1336 ("diet") appears 1 time.
Word 1443 ("start") appears 1 time.
Word 1844 ("diseas") appears 1 time.
Word 1888 ("free") appears 1 time.
Word 3103 ("gluten") appears 2 time.
Word 4419 ("intoler") appears 1 time.


In [17]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [18]:
corpus_tfidf = tfidf[bow_corpus]

In [19]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.7196780521769552), (1, 0.47007123578090043), (2, 0.5109760605999079)]


In [34]:
print("Running LDA using Bag of Words")
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=25, id2word=dictionary, passes=2, workers=2)
lda_model.save('trained_model/lda_train_bow.model')
print("LDA Training - Complete")

Running LDA using Bag of Words


TypeError: write() argument must be str, not bytes

In [21]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.126*"world" + 0.069*"histori" + 0.049*"year" + 0.025*"ii" + 0.024*"faith" + 0.023*"centuri" + 0.022*"war" + 0.019*"new" + 0.018*"ancient" + 0.018*"simpl"
Topic: 1 
Words: 0.131*"la" + 0.060*"el" + 0.040*"del" + 0.039*"le" + 0.030*"come" + 0.026*"wild" + 0.025*"break" + 0.024*"los" + 0.019*"daughter" + 0.016*"princess"
Topic: 2 
Words: 0.245*"book" + 0.087*"seri" + 0.022*"recip" + 0.021*"kid" + 0.020*"age" + 0.017*"queen" + 0.016*"child" + 0.016*"free" + 0.012*"alpha" + 0.012*"easi"
Topic: 3 
Words: 0.100*"time" + 0.057*"way" + 0.028*"garden" + 0.026*"stone" + 0.025*"kill" + 0.024*"old" + 0.020*"long" + 0.020*"seven" + 0.017*"cook" + 0.017*"troubl"
Topic: 4 
Words: 0.076*"day" + 0.055*"black" + 0.042*"citi" + 0.032*"guid" + 0.031*"end" + 0.027*"white" + 0.022*"human" + 0.019*"build" + 0.018*"handbook" + 0.017*"color"
Topic: 5 
Words: 0.069*"american" + 0.052*"rise" + 0.046*"king" + 0.037*"saga" + 0.034*"soul" + 0.027*"summer" + 0.026*"land" + 0.019*"passion" + 0.018*"

In [22]:
print("Running LDA using TF-IDF")
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=25, id2word=dictionary, passes=2, workers=4)
print("LDA Training - Complete")

Running LDA using TF-IDF
LDA Training - Complete


In [23]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.054*"die" + 0.037*"der" + 0.023*"miss" + 0.021*"son" + 0.018*"das" + 0.017*"من" + 0.016*"wish" + 0.015*"im" + 0.015*"grace" + 0.015*"des"
Topic: 1 Word: 0.028*"road" + 0.022*"old" + 0.022*"mr" + 0.019*"cowboy" + 0.019*"demon" + 0.017*"father" + 0.016*"hors" + 0.016*"awaken" + 0.016*"affair" + 0.014*"north"
Topic: 2 Word: 0.047*"death" + 0.034*"come" + 0.026*"go" + 0.019*"dont" + 0.019*"lord" + 0.018*"home" + 0.018*"water" + 0.016*"travel" + 0.015*"let" + 0.014*"day"
Topic: 3 Word: 0.038*"lose" + 0.035*"blood" + 0.033*"love" + 0.027*"ladi" + 0.020*"book" + 0.018*"walk" + 0.018*"sky" + 0.017*"time" + 0.016*"long" + 0.014*"marriag"
Topic: 4 Word: 0.027*"stori" + 0.023*"write" + 0.022*"short" + 0.021*"daughter" + 0.018*"letter" + 0.017*"best" + 0.016*"fiction" + 0.015*"hope" + 0.014*"select" + 0.014*"scienc"
Topic: 5 Word: 0.052*"night" + 0.029*"men" + 0.025*"friend" + 0.024*"wolf" + 0.022*"los" + 0.021*"brother" + 0.020*"kill" + 0.016*"rock" + 0.014*"hunt" + 0.014*"wind"


In [24]:
print("Performance evaluation by classifying sample document using LDA TF-IDF model")
for index, score in sorted(lda_model_tfidf[bow_corpus[4130]], key=lambda tup: -1*tup[1]):
    print(index)
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index)))

Performance evaluation by classifying sample document using LDA TF-IDF model
10

Score: 0.5736227035522461	 
Topic: 0.035*"boy" + 0.030*"know" + 0.028*"summer" + 0.023*"doctor" + 0.021*"tree" + 0.017*"need" + 0.016*"troubl" + 0.016*"color" + 0.015*"sleep" + 0.015*"hot"
16

Score: 0.1158323809504509	 
Topic: 0.086*"la" + 0.032*"del" + 0.029*"thing" + 0.014*"revolut" + 0.013*"final" + 0.012*"saint" + 0.012*"el" + 0.012*"horror" + 0.011*"fate" + 0.011*"happen"
20

Score: 0.11341923475265503	 
Topic: 0.040*"star" + 0.030*"break" + 0.023*"devil" + 0.022*"want" + 0.019*"box" + 0.019*"set" + 0.015*"case" + 0.015*"haunt" + 0.013*"seri" + 0.013*"train"
19

Score: 0.11310765892267227	 
Topic: 0.030*"vol" + 0.030*"bear" + 0.030*"wild" + 0.023*"like" + 0.022*"garden" + 0.021*"island" + 0.020*"tale" + 0.020*"princ" + 0.019*"season" + 0.016*"fairi"


In [25]:
print("Testing model on unseen document")
unseen_document = "The Free Voice"
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
print(lda_model[bow_vector])
print("________________________________________________________________________________________________")
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print(index)
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index)))

Testing model on unseen document
[(0, 0.013333629), (1, 0.013333629), (2, 0.346664), (3, 0.013333629), (4, 0.013333629), (5, 0.013333629), (6, 0.013333629), (7, 0.013333629), (8, 0.013333629), (9, 0.013333629), (10, 0.013333629), (11, 0.013333629), (12, 0.013333629), (13, 0.013333629), (14, 0.013333629), (15, 0.013333629), (16, 0.013333629), (17, 0.013333629), (18, 0.3466625), (19, 0.013333629), (20, 0.013333629), (21, 0.013333629), (22, 0.013333629), (23, 0.013333629), (24, 0.013333629)]
________________________________________________________________________________________________
2
Score: 0.3466639816761017	 Topic: 0.245*"book" + 0.087*"seri" + 0.022*"recip" + 0.021*"kid" + 0.020*"age" + 0.017*"queen" + 0.016*"child" + 0.016*"free" + 0.012*"alpha" + 0.012*"easi"
18
Score: 0.3466625511646271	 Topic: 0.069*"live" + 0.045*"women" + 0.038*"work" + 0.035*"woman" + 0.028*"step" + 0.026*"devil" + 0.026*"truth" + 0.024*"letter" + 0.024*"second" + 0.022*"classic"
0
Score: 0.0133336288854479

In [53]:
print("Testing model on unseen document")
conn = sqlite3.connect('nlp_learn.db')
filename = "trained_model/lda_train_bow.model"
lda_model = gensim.models.LdaModel.load(filename)
query = "SELECT id, title FROM books"
result_set = conn.execute(query).fetchall()
for row in result_set:
    unseen_document = row[1]
    bow_vector = dictionary.doc2bow(preprocess(unseen_document))
    groups = []
    group_count = 0
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        groups.append(str(index))
    query = "UPDATE books SET groups='{}' WHERE id={}".format(", ".join(groups[:5]), row[0])
    print("Updated id = {}".format(row[0]))
    conn.execute(query)
    conn.commit()

conn.close()
    
#         print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index)))

Testing model on unseen document
Updated id = 1
Updated id = 2
Updated id = 3
Updated id = 4
Updated id = 5
Updated id = 6
Updated id = 7
Updated id = 8
Updated id = 9
Updated id = 10
Updated id = 11
Updated id = 12
Updated id = 13
Updated id = 14
Updated id = 15
Updated id = 16
Updated id = 17
Updated id = 18
Updated id = 19
Updated id = 20
Updated id = 21
Updated id = 22
Updated id = 23
Updated id = 24
Updated id = 25
Updated id = 26
Updated id = 27
Updated id = 28
Updated id = 29
Updated id = 30
Updated id = 31
Updated id = 32
Updated id = 33
Updated id = 34
Updated id = 35
Updated id = 36
Updated id = 37
Updated id = 38
Updated id = 39
Updated id = 40
Updated id = 41
Updated id = 42
Updated id = 43
Updated id = 44
Updated id = 45
Updated id = 46
Updated id = 47
Updated id = 48
Updated id = 49
Updated id = 50
Updated id = 51
Updated id = 52
Updated id = 53
Updated id = 54
Updated id = 55
Updated id = 56
Updated id = 57
Updated id = 58
Updated id = 59
Updated id = 60
Updated id = 61


Updated id = 625
Updated id = 626
Updated id = 627
Updated id = 628
Updated id = 629
Updated id = 630
Updated id = 631
Updated id = 632
Updated id = 633
Updated id = 634
Updated id = 635
Updated id = 636
Updated id = 637
Updated id = 638
Updated id = 639
Updated id = 640
Updated id = 641
Updated id = 642
Updated id = 643
Updated id = 644
Updated id = 645
Updated id = 646
Updated id = 647
Updated id = 648
Updated id = 649
Updated id = 650
Updated id = 651
Updated id = 652
Updated id = 653
Updated id = 654
Updated id = 655
Updated id = 656
Updated id = 657
Updated id = 658
Updated id = 659
Updated id = 660
Updated id = 661
Updated id = 662
Updated id = 663
Updated id = 664
Updated id = 665
Updated id = 666
Updated id = 667
Updated id = 668
Updated id = 669
Updated id = 670
Updated id = 671
Updated id = 672
Updated id = 673
Updated id = 674
Updated id = 675
Updated id = 676
Updated id = 677
Updated id = 678
Updated id = 679
Updated id = 680
Updated id = 681
Updated id = 682
Updated id = 6

Updated id = 1102
Updated id = 1103
Updated id = 1104
Updated id = 1105
Updated id = 1106
Updated id = 1107
Updated id = 1108
Updated id = 1109
Updated id = 1110
Updated id = 1111
Updated id = 1112
Updated id = 1113
Updated id = 1114
Updated id = 1115
Updated id = 1116
Updated id = 1117
Updated id = 1118
Updated id = 1119
Updated id = 1120
Updated id = 1121
Updated id = 1122
Updated id = 1123
Updated id = 1124
Updated id = 1125
Updated id = 1126
Updated id = 1127
Updated id = 1128
Updated id = 1129
Updated id = 1130
Updated id = 1131
Updated id = 1132
Updated id = 1133
Updated id = 1134
Updated id = 1135
Updated id = 1136
Updated id = 1137
Updated id = 1138
Updated id = 1139
Updated id = 1140
Updated id = 1141
Updated id = 1142
Updated id = 1143
Updated id = 1144
Updated id = 1145
Updated id = 1146
Updated id = 1147
Updated id = 1148
Updated id = 1149
Updated id = 1150
Updated id = 1151
Updated id = 1152
Updated id = 1153
Updated id = 1154
Updated id = 1155
Updated id = 1156
Updated id

Updated id = 1558
Updated id = 1559
Updated id = 1560
Updated id = 1561
Updated id = 1562
Updated id = 1563
Updated id = 1564
Updated id = 1565
Updated id = 1566
Updated id = 1567
Updated id = 1568
Updated id = 1569
Updated id = 1570
Updated id = 1571
Updated id = 1572
Updated id = 1573
Updated id = 1574
Updated id = 1575
Updated id = 1576
Updated id = 1577
Updated id = 1578
Updated id = 1579
Updated id = 1580
Updated id = 1581
Updated id = 1582
Updated id = 1583
Updated id = 1584
Updated id = 1585
Updated id = 1586
Updated id = 1587
Updated id = 1588
Updated id = 1589
Updated id = 1590
Updated id = 1591
Updated id = 1592
Updated id = 1593
Updated id = 1594
Updated id = 1595
Updated id = 1596
Updated id = 1597
Updated id = 1598
Updated id = 1599
Updated id = 1600
Updated id = 1601
Updated id = 1602
Updated id = 1603
Updated id = 1604
Updated id = 1605
Updated id = 1606
Updated id = 1607
Updated id = 1608
Updated id = 1609
Updated id = 1610
Updated id = 1611
Updated id = 1612
Updated id

Updated id = 2015
Updated id = 2016
Updated id = 2017
Updated id = 2018
Updated id = 2019
Updated id = 2020
Updated id = 2021
Updated id = 2022
Updated id = 2023
Updated id = 2024
Updated id = 2025
Updated id = 2026
Updated id = 2027
Updated id = 2028
Updated id = 2029
Updated id = 2030
Updated id = 2031
Updated id = 2032
Updated id = 2033
Updated id = 2034
Updated id = 2035
Updated id = 2036
Updated id = 2037
Updated id = 2038
Updated id = 2039
Updated id = 2040
Updated id = 2041
Updated id = 2042
Updated id = 2043
Updated id = 2044
Updated id = 2045
Updated id = 2046
Updated id = 2047
Updated id = 2048
Updated id = 2049
Updated id = 2050
Updated id = 2051
Updated id = 2052
Updated id = 2053
Updated id = 2054
Updated id = 2055
Updated id = 2056
Updated id = 2057
Updated id = 2058
Updated id = 2059
Updated id = 2060
Updated id = 2061
Updated id = 2062
Updated id = 2063
Updated id = 2064
Updated id = 2065
Updated id = 2066
Updated id = 2067
Updated id = 2068
Updated id = 2069
Updated id

...

Updated id = 87746
Updated id = 87747
Updated id = 87748
Updated id = 87749
Updated id = 87750
Updated id = 87751
Updated id = 87752
Updated id = 87753
Updated id = 87754
Updated id = 87755
Updated id = 87756
Updated id = 87757
Updated id = 87758
Updated id = 87759
Updated id = 87760
Updated id = 87761
Updated id = 87762
Updated id = 87763
Updated id = 87764
Updated id = 87765
Updated id = 87766
Updated id = 87767
Updated id = 87768
Updated id = 87769
Updated id = 87770
Updated id = 87771
Updated id = 87772
Updated id = 87773
Updated id = 87774
Updated id = 87775
Updated id = 87776
Updated id = 87777
Updated id = 87778
Updated id = 87779
Updated id = 87780
Updated id = 87781
Updated id = 87782
Updated id = 87783
Updated id = 87784
Updated id = 87785
Updated id = 87786
Updated id = 87787
Updated id = 87788
Updated id = 87789
Updated id = 87790
Updated id = 87791
Updated id = 87792
Updated id = 87793
Updated id = 87794
Updated id = 87795
Updated id = 87796
Updated id = 87797
Updated id =

Updated id = 88178
Updated id = 88179
Updated id = 88180
Updated id = 88181
Updated id = 88182
Updated id = 88183
Updated id = 88184
Updated id = 88185
Updated id = 88186
Updated id = 88187
Updated id = 88188
Updated id = 88189
Updated id = 88190
Updated id = 88191
Updated id = 88192
Updated id = 88193
Updated id = 88194
Updated id = 88195
Updated id = 88196
Updated id = 88197
Updated id = 88198
Updated id = 88199
Updated id = 88200
Updated id = 88201
Updated id = 88202
Updated id = 88203
Updated id = 88204
Updated id = 88205
Updated id = 88206
Updated id = 88207
Updated id = 88208
Updated id = 88209
Updated id = 88210
Updated id = 88211
Updated id = 88212
Updated id = 88213
Updated id = 88214
Updated id = 88215
Updated id = 88216
Updated id = 88217
Updated id = 88218
Updated id = 88219
Updated id = 88220
Updated id = 88221
Updated id = 88222
Updated id = 88223
Updated id = 88224
Updated id = 88225
Updated id = 88226
Updated id = 88227
Updated id = 88228
Updated id = 88229
Updated id =

Updated id = 88612
Updated id = 88613
Updated id = 88614
Updated id = 88615
Updated id = 88616
Updated id = 88617
Updated id = 88618
Updated id = 88619
Updated id = 88620
Updated id = 88621
Updated id = 88622
Updated id = 88623
Updated id = 88624
Updated id = 88625
Updated id = 88626
Updated id = 88627
Updated id = 88628
Updated id = 88629
Updated id = 88630
Updated id = 88631
Updated id = 88632
Updated id = 88633
Updated id = 88634
Updated id = 88635
Updated id = 88636
Updated id = 88637
Updated id = 88638
Updated id = 88639
Updated id = 88640
Updated id = 88641
Updated id = 88642
Updated id = 88643
Updated id = 88644
Updated id = 88645
Updated id = 88646
Updated id = 88647
Updated id = 88648
Updated id = 88649
Updated id = 88650
Updated id = 88651
Updated id = 88652
Updated id = 88653
Updated id = 88654
Updated id = 88655
Updated id = 88656
Updated id = 88657
Updated id = 88658
Updated id = 88659
Updated id = 88660
Updated id = 88661
Updated id = 88662
Updated id = 88663
Updated id =

Updated id = 89045
Updated id = 89046
Updated id = 89047
Updated id = 89048
Updated id = 89049
Updated id = 89050
Updated id = 89051
Updated id = 89052
Updated id = 89053
Updated id = 89054
Updated id = 89055
Updated id = 89056
Updated id = 89057
Updated id = 89058
Updated id = 89059
Updated id = 89060
Updated id = 89061
Updated id = 89062
Updated id = 89063
Updated id = 89064
Updated id = 89065
Updated id = 89066
Updated id = 89067
Updated id = 89068
Updated id = 89069
Updated id = 89070
Updated id = 89071
Updated id = 89072
Updated id = 89073
Updated id = 89074
Updated id = 89075
Updated id = 89076
Updated id = 89077
Updated id = 89078
Updated id = 89079
Updated id = 89080
Updated id = 89081
Updated id = 89082
Updated id = 89083
Updated id = 89084
Updated id = 89085
Updated id = 89086
Updated id = 89087
Updated id = 89088
Updated id = 89089
Updated id = 89090
Updated id = 89091
Updated id = 89092
Updated id = 89093
Updated id = 89094
Updated id = 89095
Updated id = 89096
Updated id =

KeyboardInterrupt: 

In [42]:
a = ["1", "3", "4", "5"]
print(", ".join(a))

1, 3, 4, 5


In [ ]:
a = "Salman's bag"
a.replace("'", "")